<a href="https://colab.research.google.com/github/RiazMahmood6/PHero-tube/blob/main/0_5_Stable_Diffusion_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate torch torchvision matplotlib

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch

# Load the lightweight model
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-base",  # Smaller model
    torch_dtype=torch.float16,
    revision="fp16"
).to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:221: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2-base via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(


...

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [ ]:
from PIL import Image
from google.colab import files

# Upload an image
uploaded = files.upload()
input_image = list(uploaded.keys())[0]

# Load the image
init_image = Image.open(input_image)
init_image = init_image.convert("RGB")
init_image = init_image.resize((512, 512))  # Resize to 512x512


Saving 4.png to 4 (1).png


In [ ]:
print(type(init_image))  # Should output: <class 'PIL.Image.Image'>

<class 'PIL.Image.Image'>


In [ ]:
prompt = " a humorous, exaggerated caricature of person. His face has oversized, intense eyes with one eyebrow raised high and a huge, toothy grin. His muscular body is exaggerated to immense proportions. He wears a tiny, comically tight swimsuit and holds an enormous surfboard. The background features a lively beach scene with giant waves, amused beachgoers, and playful cartoon crabs."  # Modify as needed
strength = 0.75  # Controls how much the image is transformed
guidance_scale = 7.5  # Controls adherence to the prompt

# Run the img2img process
result = pipe(
    prompt=prompt,
    image=init_image,  # Note: Use "image" instead of "init_image" if that's what your pipeline expects
    strength=strength,
    guidance_scale=guidance_scale
).images[0]


# Save and display the result
result.save("output.png")
result.show()

  0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
from google.colab import files

files.download("output.png")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **FineTune**

**new**

In [ ]:
# Install required packages
!pip install diffusers transformers accelerate torch torchvision matplotlib
# Import required libraries
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image
import os

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Specify the path to your dataset
dataset_path = '/content/drive/dreambooth_training'

# Load the lightweight model
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-base",  # Smaller model
    torch_dtype=torch.float16,
    revision="fp16"
)

# Function to process images in the dataset
def process_images(dataset_path):
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                image = image.convert("RGB")
                image = image.resize((512, 512))  # Resize to 512x512
                yield image_path, image

# Process images in the dataset
for image_path, image in process_images(dataset_path):
    prompt = "a humorous, exaggerated caricature of person. His face has oversized, intense eyes with one eyebrow raised high and a huge, toothy grin. His muscular body is exaggerated to immense proportions. He wears a tiny, comically tight swimsuit and holds an enormous surfboard. The background features a lively beach scene with giant waves, amused beachgoers, and playful cartoon crabs."  # Modify as needed
    strength = 0.75  # Controls how much the image is transformed
    guidance_scale = 7.5  # Controls adherence to the prompt

    # Run the img2img process
    result = pipe(
        prompt=prompt,
        image=image,  # Note: Use "image" instead of "init_image" if that's what your pipeline expects
        strength=strength,
        guidance_scale=guidance_scale
    ).images[0]

    # Save and display the result
    result.save(os.path.join('/content/drive/dreambooth_results', f"output_{image_path.split('/')[-1]}"))
    result.show()

# Unmount Google Drive
drive.unmount()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:221: FutureWarning: You are loading the variant fp16 from stabilityai/stable-diffusion-2-base via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-base/snapshots/1cb61502fc8b634cdb04e7cd69e06051a728bedf/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/37 [00:00<?, ?it/s]